# 1.Correct Implementation

In [10]:
%%writefile correct.py
def left_rotate(s: str, d: int) -> str:
    """Left rotate string s by d characters."""
    d = d % len(s) if s else 0   # handle d > len(s)
    return s[d:] + s[:d]

Overwriting correct.py


In [11]:
from correct import left_rotate
print(left_rotate("python", 2))   # thonpy
print(left_rotate("bigdata", 3))  # databig
print(left_rotate("hadoop", 1))   # adooph

thonpy
databig
adooph


# 2.Buggy Implementation

In [12]:
%%writefile buggy.py
def left_rotate(s: str, d: int) -> str:
    """BUGGY: performs right rotation instead of left rotation."""
    if not s:
        return s
    d = d % len(s)
    return s[-d:] + s[:-d]   # ❌ wrong direction

Overwriting buggy.py


In [13]:
from buggy import left_rotate
print(left_rotate("python", 2))  # ❌ onpyth (should be thonpy)

onpyth


# 3.LLM Testing(pytest)

In [14]:
%%writefile test_llm_generated.py
import pytest
from buggy import left_rotate

def test_basic_examples():
    assert left_rotate("python", 2) == "thonpy"
    assert left_rotate("bigdata", 3) == "databig"
    assert left_rotate("hadoop", 1) == "adooph"

def test_extra_cases():
    assert left_rotate("abc", 0) == "abc"
    assert left_rotate("abc", 3) == "abc"
    assert left_rotate("abc", 5) == "cab"  # same as rotating by 2
    assert left_rotate("", 5) == ""

Overwriting test_llm_generated.py


In [15]:
!pytest -q test_llm_generated.py --maxfail=1

F
================================== FAILURES ===================================
_____________________________ test_basic_examples _____________________________

    def test_basic_examples():
>       assert left_rotate("python", 2) == "thonpy"
E       AssertionError: assert 'onpyth' == 'thonpy'
E         
E         - thonpy
E         + onpyth

test_llm_generated.py:5: AssertionError
=========================== short test summary info ===========================
FAILED test_llm_generated.py::test_basic_examples - AssertionError: assert 'onpyth' == 'thonpy'
!!!!!!!!!!!!!!!!!!!!!!!!!! stopping after 1 failures !!!!!!!!!!!!!!!!!!!!!!!!!!
1 failed in 0.62s


# 4 – Human property-based tests (hypothesis)
We can define properties for string rotation:
Length is preserved.
Rotating twice is equivalent to rotating once by sum of both (mod n).
Rotating by len(s) gives the same string.

In [16]:
%%writefile test_properties.py
from hypothesis import given, strategies as st
from buggy import left_rotate

@given(st.text(), st.integers(min_value=0, max_value=100))
def test_length_preserved(s, d):
    out = left_rotate(s, d)
    assert len(out) == len(s)

@given(st.text(min_size=1), st.integers(min_value=0, max_value=50))
def test_rotation_equivalence(s, d):
    out1 = left_rotate(left_rotate(s, d), d)
    out2 = left_rotate(s, 2*d)
    assert out1 == out2  # rotation composition property

@given(st.text(min_size=1))
def test_full_rotation_identity(s):
    out = left_rotate(s, len(s))
    assert out == s

Overwriting test_properties.py


In [17]:
!pytest -q test_properties.py --maxfail=1

...                                                                      [100%]
3 passed in 1.13s


# 5 – Save showdown results

In [18]:
import json, pytest

llm_exit = pytest.main(["-q", "test_llm_generated.py", "--maxfail=1"])
human_exit = pytest.main(["-q", "test_properties.py", "--maxfail=1"])

results = {
    "found_by_llm": (llm_exit != 0),
    "found_by_human": (human_exit != 0)
}

with open("results.json", "w") as f:
    json.dump(results, f, indent=2)

results

F
================================== FAILURES ===================================
_____________________________ test_basic_examples _____________________________

    def test_basic_examples():
>       assert left_rotate("python", 2) == "thonpy"
E       AssertionError: assert 'onpyth' == 'thonpy'
E         
E         - thonpy
E         + onpyth

test_llm_generated.py:5: AssertionError
=========================== short test summary info ===========================
FAILED test_llm_generated.py::test_basic_examples - AssertionError: assert 'onpyth' == 'thonpy'
!!!!!!!!!!!!!!!!!!!!!!!!!! stopping after 1 failures !!!!!!!!!!!!!!!!!!!!!!!!!!
1 failed in 0.03s
...                                                                      [100%]
3 passed in 0.35s


{'found_by_llm': True, 'found_by_human': False}